In [9]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd



In [ ]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database
conn = create_engine(string).connect()

df = pd.read_sql(
    """
    select date, ticker, ret, roeq, mom12m
    from data
    where date>='2010-01' and date<='2017-12'
    """, 
    conn
)
df = df.dropna()
df = df.set_index("date")

In [10]:
from pandas_datareader import DataReader as pdr

mkt = pdr(
  "F-F_Research_Data_Factors",
  "famafrench",
  start="2010-01",
  end="2017-12"
)
mkt = mkt[0] / 100
mkt.index = mkt.index.astype(str)
mkt["mkt"] = mkt["Mkt-RF"] + mkt["RF"]

df = df.join(mkt, how="left")

In [12]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import LinearRegression

model = TransformedTargetRegressor(
    transformer=QuantileTransformer(
      output_distribution="normal"
    ),
    regressor=LinearRegression()
)

X = df[["roeq", "mom12m"]]
y = df.ret - df.mkt
model.fit(X, y)

SyntaxError: invalid syntax (901135457.py, line 1)